In [52]:
import urllib.request,re, os
import pandas as pd
from bs4 import BeautifulSoup

#Use BeautifulSoup to scrap data from the wiki page
# read the page content to the_page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.Request(url)
with urllib.request.urlopen(req) as response:
     the_page = response.read()     
soup=BeautifulSoup(the_page,'html.parser')

#parse each table cells into list : lst_origin_data
lst_origin_data = []
table = soup.find('table', attrs={'class':'wikitable'})
table_body = table.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    lst_origin_data.append([ele for ele in cols if ele]) # Get rid of empty values

#process the data with Not assigned value accordingly and store into dictionary with keys : dict_valid_data
dict_valid_data = dict()
for index,cell in enumerate(lst_origin_data):
    if cell:
        t_postcode=cell[0]
        t_borough=cell[1]
        t_neighborhood=cell[2]
        if not t_borough == "Not assigned":           #only process data with borough assigned
            if t_neighborhood == "Not assigned":      #assign borough to neighborhood if neighborhood is not assigned
                t_neighborhood = t_borough
            dict_valid_data.update({
                t_postcode:{
                    'Borough':t_borough,
                    'Neighborhood':(dict_valid_data[t_postcode]['Neighborhood']+
                                    ','+t_neighborhood 
                                    if t_postcode in dict_valid_data 
                                    else t_neighborhood)
                }})                                   #update neighborhood for multiple neighborhoods
            

print(len(dict_valid_data))
#dict_valid_data


103


In [53]:
#conver the dictionary dict_valid_data to DataFrame: df
df=pd.DataFrame.from_dict(dict_valid_data,orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index':'PostCode'},inplace=True)
print(df.shape)
df.head()

(103, 3)


,PostCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
